## 학습 6

## 텐서플로우 학습 6(File load in Tensorflow and Queue Thread)

**출처 ** <br>
- https://www.youtube.com/watch?v=rxgvKa0MdK4

In [1]:
import tensorflow as tf

<br><br>** 아래와 같은 내용으로 test.csv 파일을 하나 생성한다. **

10, 20, 30, 40, 50, <br>
1, 2, 3, 4, 5 <br>
2, 4, 6, 8, 10<br><br>

** 해당 파일을 생성한 경로를 아래와 같은 함수를 통하여 넣어준다 **

해당 파일 queue에 넣어준다. 레코드(가로) 기준으로

In [2]:
filename_queue = tf.train.string_input_producer(['data/test.csv'])
filename_queue

<br>** reader 는 레코드를 읽는 역할을 한다.  **

In [3]:
reader = tf.TextLineReader()
reader

In [4]:
key, value = reader.read(filename_queue)
print(key, '  /  ', value)

Tensor("ReaderRead:0", shape=(), dtype=string)   /   Tensor("ReaderRead:1", shape=(), dtype=string)


<br>
** 값이 비어 있을 때, 넣어주는 record_defaults **

레코드에서 값이 비어있는 컬럼이 있으면, 해당 위치의 값을 채우는 역할! 

In [5]:
record_defaults = [[1],[1],[1],[1],[1]]

<br>
** 레코드를 읽어서 각각의 컬럼 변수에 넣어준다. **


In [6]:
col1, col2, col3, col4, col5 = tf.decode_csv(value, record_defaults=record_defaults)

In [7]:
col1

<tf.Tensor 'DecodeCSV:0' shape=() dtype=int32>

<br>
** 분리된 컬럼을 하나로 합친다  **


In [8]:
features = tf.pack([col1, col2, col3, col4, col5])
features

<tf.Tensor 'pack:0' shape=(5,) dtype=int32>

<br>
** 세션을 진행한다. with 을 사용하면 따로 sess를 종료하지 않아줘도 들여쓰기가 끝나는 구간에서 종료한다.  **

- queue_runner는 실제 thread queue를 실행시켜준다. 
- Coordinator()는 thread queue 의 on / off 스위치 역할 

In [12]:
with tf.Session() as sess: 
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord= coord)
    
    for i in range(3):
        example, label = sess.run([features, col5]) 
        print (example)
        print (label)
        
    coord.request_stop()
    coord.join(threads)

[10 20 30 40 50 60 70 80 90]
50
[1 2 3 4 5 6 7 8 9]
5
[ 2  4  6  8 10 12 14 16 18]
10


In [10]:
print( 'example : ', example )
print( 'label : ', label )

example :  [ 2  4  6  8 10]
label :  10


### pack() 은 여러 컬럼을 하나로 합쳐주는 역할을 한다 

** 아래와 같은 내용으로 test2.csv 파일을 하나 생성한다. **

10, 20, 30, 40, 50, 60, 70, 80, 90 <br>
1, 2, 3, 4, 5, 6 ,7 , 8, 9 <br>
2, 4, 6, 8, 10, 12, 14, 16 , 18<br>

In [11]:
import tensorflow as tf

filename_queue = tf.train.string_input_producer(['data/test2.csv'])
reader = tf.TextLineReader() 
key, value = reader.read(filename_queue) 
record_defaults = [[1],[1],[1],[1],[1],[1],[1],[1],[1]]
col1, col2, col3, col4, col5, col6, col7, col8, col9 = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.pack([col1, col2, col3, col4, col5, col6, col7, col8, col9])

with tf.Session() as sess: 
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord= coord)
    
    for i in range(3):
        example, label = sess.run([features, col9])
        
        print (example)
        print (label)
        
    coord.request_stop()
    coord.join(threads)

[10 20 30 40 50 60 70 80 90]
90
[1 2 3 4 5 6 7 8 9]
9
[ 2  4  6  8 10 12 14 16 18]
18


<br><br><br><br><br><br>